In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import time
import random

In [2]:
def load_data(filename):
    data_list = []
    with open(filename, 'r') as data:
        for line in tqdm(data):
            user_id, item_id, rating, timestamp = line.split('\t')
            data_list.append([int(user_id), int(item_id), int(rating)])
            
    print('data loading complete')
    return data_list

In [4]:
train_data = load_data('ml-100k/u1.base')
test_data = load_data('ml-100k/u1.test')


data loading complete



data loading complete


# SVD

In [16]:
class SVD():

    # Initializing the user-movie rating matrix, no. of latent features
    def __init__(self, ratings, num_users, num_items, dim):
        self.ratings = ratings
        self.num_users, self.num_items = num_users, num_items
        self.dim = dim
        self.mu = np.mean([i[2] for i in ratings])
        
        self.P = np.random.normal(scale=1./self.dim, size=(self.num_users, self.dim))
        self.Q = np.random.normal(scale=1./self.dim, size=(self.num_items, self.dim))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        
    # Initializing user-feature and movie-feature matrix 
    def train(self,  lr_rate, weight_decay, thresold, epoches):
        self.lr_rate = lr_rate
        self.weight_decay = weight_decay
        
        #delta = np.inf
        #idx = 1
        #past_loss = np.inf
        print('Begin training')
        start = time.process_time()
        #while delta > thresold :
        #curren_loss = np.inf
        #while curren_loss <= past_loss:
        for i in tqdm(range(epoches)):
            #past_loss = curren_loss
            random.shuffle(self.ratings)
            for user, item, rating in self.ratings:
                
                predict = self.get_rating(user, item)
                delta = max(self.sgd(user, item, rating, predict))
            curren_loss = self.rmse()
            #print('epoch : {0}  loss = '.format(idx), curren_loss)
            print('epoch : {0} / {1} loss = '.format(i+1, epoches), curren_loss)
            #idx+=1
            self.evaluate(test_data)
            #print(delta)
        print('Training complete \n\nTime past : %.2f seconds'%(time.process_time()-start))
        
    def evaluate(self, test_data):
        predicted = self.full_matrix()
        error = 0
        for i, j, rating in test_data:
             error += pow(rating - predicted[i-1, j-1], 2)
           
        print('RMSE of testing data : %.4f'%np.sqrt(error/len(test_data))) 
    # Computing total mean squared error
    def rmse(self):
        
        predicted = self.full_matrix()
        error = 0
        for i, j, rating in self.ratings:
             error += pow(rating - predicted[i-1, j-1], 2)
           
        return np.sqrt(error/len(self.ratings))

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self, i, j, true, predict):
        
       
        e = (true - predict)

        self.b_u[i-1] += self.lr_rate * (e - self.weight_decay * self.b_u[i-1])
        self.b_i[j-1] += self.lr_rate * (e - self.weight_decay * self.b_i[j-1])

        tmp = e * self.P[i-1, :] - self.weight_decay * self.Q[j-1,:]
        
        self.P[i-1, :] += self.lr_rate * (e * self.Q[j-1, :] - self.weight_decay * self.P[i-1,:])
        
        self.Q[j-1, :] += self.lr_rate * (tmp)
   
        return [np.abs(self.lr_rate * (e - self.weight_decay * self.b_u[i-1])),
                np.abs(self.lr_rate * (e - self.weight_decay * self.b_i[j-1])),
                max(np.abs(self.lr_rate * (e * self.Q[j-1, :] - self.weight_decay * self.P[i-1,:]))),
                max(np.abs(self.lr_rate * (tmp)))
               ]
    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.mu + self.b_u[i-1] + self.b_i[j-1] + self.P[i-1, :].dot(self.Q[j-1, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return self.mu + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [17]:
len_u = 943
len_v = 1682
#batch_size = 10
lr_rate = 0.005
weight_decay= 0.1#0.05
thresold = 1e-5
epoches = 100
dim = 100
######################################
model_svd = SVD(ratings=train_data, num_users=len_u, num_items=len_v, dim=dim)
model_svd.train(lr_rate=lr_rate, weight_decay=weight_decay, thresold=thresold, epoches=epoches)
model_svd.evaluate(test_data)

Begin training


epoch : 1 / 100 loss =  0.9951667017590311
RMSE of testing data : 1.0318
epoch : 2 / 100 loss =  0.9643995710763887
RMSE of testing data : 1.0011
epoch : 3 / 100 loss =  0.9498655690268534
RMSE of testing data : 0.9874
epoch : 4 / 100 loss =  0.9415426441182875
RMSE of testing data : 0.9799
epoch : 5 / 100 loss =  0.9358128028601135
RMSE of testing data : 0.9748
epoch : 6 / 100 loss =  0.931730829322305
RMSE of testing data : 0.9714
epoch : 7 / 100 loss =  0.9286188223536308
RMSE of testing data : 0.9687
epoch : 8 / 100 loss =  0.9262024579565997
RMSE of testing data : 0.9664
epoch : 9 / 100 loss =  0.9242999899070653
RMSE of testing data : 0.9649
epoch : 10 / 100 loss =  0.9228169965141758
RMSE of testing data : 0.9635
epoch : 11 / 100 loss =  0.9214873319885876
RMSE of testing data : 0.9625
epoch : 12 / 100 loss =  0.9205405410966926
RMSE of testing data : 0.9614
epoch : 13 / 100 loss =  0.919497478125933
RMSE of testing data : 0.9606
epoch : 14 / 100 loss =  0.918738058097417
RMSE o

In [18]:
pre = model_svd.full_matrix()
for i, j, rating in train_data[:20]:
    print('real :', rating, 'predict : %.2f'%pre[i-1, j-1])

real : 3 predict : 3.16
real : 4 predict : 4.73
real : 2 predict : 3.30
real : 3 predict : 3.42
real : 3 predict : 2.93
real : 3 predict : 4.25
real : 2 predict : 2.83
real : 5 predict : 3.67
real : 5 predict : 4.36
real : 1 predict : 2.69
real : 4 predict : 3.85
real : 4 predict : 2.70
real : 5 predict : 4.39
real : 5 predict : 4.40
real : 1 predict : 0.92
real : 4 predict : 3.73
real : 5 predict : 3.70
real : 1 predict : 2.70
real : 3 predict : 3.76
real : 4 predict : 3.98


In [19]:
for i, j, rating in test_data[:20]:
    print('real :', rating, 'predict : %.2f'%pre[i-1, j-1])

real : 5 predict : 3.70
real : 3 predict : 3.98
real : 5 predict : 4.55
real : 5 predict : 4.03
real : 3 predict : 3.25
real : 4 predict : 3.79
real : 4 predict : 4.31
real : 3 predict : 3.63
real : 2 predict : 3.03
real : 3 predict : 3.63
real : 4 predict : 3.61
real : 2 predict : 2.47
real : 4 predict : 3.39
real : 5 predict : 3.16
real : 4 predict : 3.81
real : 3 predict : 3.44
real : 4 predict : 3.43
real : 3 predict : 3.01
real : 3 predict : 3.24
real : 4 predict : 4.41
